In [1]:
# using kernel deepLearn_V2

import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

import os
import pandas as pd
import time
from datetime import datetimeL
import sys
import csv
import seaborn as sns
import scipy.io

print(sys.version, "\n")

now = datetime.now()
print("last run on " + str(now))

3.6.5 |Anaconda custom (64-bit)| (default, Apr 29 2018, 16:14:56) 
[GCC 7.2.0] 

last run on 2018-06-08 14:34:04.552300


In [2]:
from sklearn.preprocessing import StandardScaler 

In [3]:
from keras.models import Sequential
from keras.layers import Dense


Using TensorFlow backend.


In [4]:
import tensorflow as tf
print("TensorFlow successfully installed.")
if tf.test.is_built_with_cuda():
    print("The installed version of TensorFlow includes GPU support.")

TensorFlow successfully installed.
The installed version of TensorFlow includes GPU support.


In [5]:
os.environ['DESKTOP_SESSION']

'ubuntu'

In [6]:
def windowsOrMacDirectories():
    """ Sets base directories for win or mac or ubuntu

       
    """
    if sys.platform.startswith('linux') or sys.platform.startswith('cygwin'):
#         # this excludes your current terminal "/dev/tty"
        DropboxDirect = os.path.join("/hdd12tb/Dropbox")
        dataDir = os.path.join("/hdd12tb/MothSimulations/")
    elif os.environ['COMPUTERNAME'] == 'SHEALMACLEARN':
        DropboxDirect = os.path.join("D:\Dropbox")
# refref: these are wrong (mac and windows)
    elif sys.platform.startswith('darwin'):
        DropboxDirect = os.path.join("/Users/cswitzer/Dropbox")
    else:
        raise EnvironmentError('Unknown computer platform')
    
    baseDir = os.getcwd()
    #dataDir = os.path.join(DropboxDirect, 'mothMachineLearning_dataAndFigs', 'Data')
    figDir = os.path.join(DropboxDirect, 'mothMachineLearning_dataAndFigs', 'Figs')
    
    assert (os.path.isdir(baseDir), 
            os.path.isdir(dataDir), 
            os.path.isdir(figDir)) == (True, 
                                       True, 
                                       True), "One or more directories don't exist"
    return baseDir, dataDir, figDir


baseDir, dataDir, figDir = windowsOrMacDirectories()


In [7]:
# define directories
print(baseDir, "\n", dataDir, "\n", figDir)

/home/cswitzer/Documents/GitRepos/MothMachineLearning 
 /hdd12tb/MothSimulations/ 
 /hdd12tb/Dropbox/mothMachineLearning_dataAndFigs/Figs


In [10]:
os.path.isfile(os.path.join(dataDir,"11c-AggressiveManeuver", "Qstore", "Qstore_1_am_con.mat"))

True

In [13]:
os.path.isfile(os.path.join(dataDir,"OtherFiles_beforeparallelization", "ValSp3_1_am_con.mat"))

True

In [73]:
# read in data
Qmat = scipy.io.loadmat(os.path.join(dataDir,"11c-AggressiveManeuver", "Qstore", "Qstore_11_am_ytf.mat"), squeeze_me=True, struct_as_record=False)
#Qmat = scipy.io.loadmat(os.path.join(dataDir,"11c-AggressiveManeuver", "Qstore", "Qstore_1_am_con.mat"))
Vmat = scipy.io.loadmat(os.path.join(dataDir,"OtherFiles_beforeparallelization", "ValSp3_1_am_con.mat"))

In [89]:
import scipy
import scipy.io as spio

def loadmat(filename):
    '''
    this function should be called instead of direct spio.loadmat
    as it cures the problem of not properly recovering python dictionaries
    from mat files. It calls the function check keys to cure all entries
    which are still mat-objects
    '''
    def _check_keys(d):
        '''
        checks if entries in dictionary are mat-objects. If yes
        todict is called to change them to nested dictionaries
        '''
        for key in d:
            if isinstance(d[key], spio.matlab.mio5_params.mat_struct):
                d[key] = _todict(d[key])
        return d

    def _todict(matobj):
        '''
        A recursive function which constructs from matobjects nested dictionaries
        '''
        d = {}
        for strg in matobj._fieldnames:
            elem = matobj.__dict__[strg]
            if isinstance(elem, spio.matlab.mio5_params.mat_struct):
                d[strg] = _todict(elem)
            elif isinstance(elem, np.ndarray):
                d[strg] = _tolist(elem)
            else:
                d[strg] = elem
        return d

    def _tolist(ndarray):
        '''
        A recursive function which constructs lists from cellarrays
        (which are loaded as numpy ndarrays), recursing into the elements
        if they contain matobjects.
        '''
        elem_list = []
        for sub_elem in ndarray:
            if isinstance(sub_elem, spio.matlab.mio5_params.mat_struct):
                elem_list.append(_todict(sub_elem))
            elif isinstance(sub_elem, np.ndarray):
                elem_list.append(_tolist(sub_elem))
            else:
                elem_list.append(sub_elem)
        return elem_list
    data = scipy.io.loadmat(filename, struct_as_record=False, squeeze_me=True)
    return _check_keys(data)

In [90]:
qstr = loadmat(os.path.join(dataDir,"11c-AggressiveManeuver", "Qstore", "Qstore_11_am_ytf.mat"))

In [92]:
qstr.keys()

dict_keys(['__header__', '__version__', '__globals__', 'Qstore'])

In [110]:
dir(qstr["Qstore"][1])

['T',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_finalize__',
 '__array_interface__',
 '__array_prepare__',
 '__array_priority__',
 '__array_struct__',
 '__array_ufunc__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__complex__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__ilshift__',
 '__imatmul__',
 '__imod__',
 '__imul__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__irshift__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lshift__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdivmod__',
 '__reduce__',
 '__reduce_e

In [125]:
dir(qstr["Qstore"][0][0]) # gives F, ICs, NewICs, alpha, tau0

['F',
 'ICs',
 'NewICs',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slotnames__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_fieldnames',
 'alpha',
 'bigQ',
 'check',
 'cost',
 'tau0']

In [128]:
qstr["Qstore"][0][0].bigQ.shape # gives x, xdot, y, ydot, theta, thetadot, etc

(100, 8)

In [134]:
qstr["Qstore"][0][0].tau0

-69642.13027859683

In [132]:
dir(qstr["Qstore"][0][0].bigQ)

['T',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_finalize__',
 '__array_interface__',
 '__array_prepare__',
 '__array_priority__',
 '__array_struct__',
 '__array_ufunc__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__complex__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__ilshift__',
 '__imatmul__',
 '__imod__',
 '__imul__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__irshift__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lshift__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdivmod__',
 '__reduce__',
 '__reduce_e

In [109]:
qstr["Qstore"][0][0].ICs

array([8.23596513e-02, 2.22430811e+00, 7.87845276e-01, 3.95213518e+00,
       8.16525851e+00, 2.22368300e+02, 2.97856355e-01, 2.58911053e+00])

In [75]:
Qmat.keys()


dict_keys(['__header__', '__version__', '__globals__', 'Qstore'])

In [76]:
Qdta = Qmat["Qstore"]

In [86]:
Qdta[0][0]

array(<scipy.io.matlab.mio5_params.mat_struct object at 0x7f44237a00b8>,
      dtype=object)

In [31]:
Qdta.shape

(10000, 20000)

In [32]:
Vmat.keys()

dict_keys(['__header__', '__version__', '__globals__', 'ValSp3'])

In [33]:
Qdta = Qmat["Qstore"]
dd = pd.DataFrame(Qdta)
dd['time'] = np.arange(0, 2, 2.0002e-04)
cols = dd.columns.tolist()
dd = dd[[cols[-1]] + cols[:-1]] 
print(dd.shape)

(10000, 20001)


In [34]:
dd['timestep'] = (np.floor(np.arange(0, 100, step = 0.01))).astype(int)
cols = dd.columns.tolist()
dd = dd[[cols[-1]] + cols[:-1]] 

In [35]:
dd.head()

,timestep,time,0,1,2,3,4,5,6,7,...,19990,19991,19992,19993,19994,19995,19996,19997,19998,19999
0,0,0.0000,0.000000,0.000000,0.785398,3.926991,0.000100,0.000100,0.000000,0.000000,...,0.000000,0.000000,0.000000e+00,0.000000,0.785398,3.926991,0.000100,0.000100,0.000000,0.000000
1,0,0.0002,0.000103,0.000041,0.785405,3.926991,1.003792,0.422649,0.048971,0.000594,...,0.012709,0.027796,4.027603e-07,-0.000124,0.785407,3.926990,-0.011122,-1.211176,0.066204,-0.005158
2,0,0.0004,0.000405,0.000172,0.785416,3.926991,1.983931,0.868754,0.057278,0.003731,...,0.032963,0.054989,-5.700732e-06,-0.000489,0.785423,3.926989,-0.049237,-2.395544,0.085917,-0.007513
3,0,0.0006,0.000906,0.000393,0.785429,3.926992,2.963612,1.315319,0.064795,0.006861,...,0.053276,0.082060,-1.955832e-05,-0.001092,0.785442,3.926987,-0.087493,-3.579753,0.105314,-0.010017
4,0,0.0008,0.001603,0.000703,0.785442,3.926994,3.943133,1.762049,0.072037,0.009950,...,0.073520,0.109017,-4.117008e-05,-0.001935,0.785465,3.926985,-0.126621,-4.763069,0.123132,-0.012591


In [ ]:
#%qtconsole

In [ ]:
cc = np.zeros(Qdta.shape).reshape((-1, 100*8))
cc.shape

In [ ]:
# another possible way to rearrange the data
stt = time.time()

ss = 8 # number of columns to repeat
cctr = 0 # output row counter


rctr = 0 # row counter for input data
    
while rctr < Qdta.shape[0]:
    
    ii = 0 # column counter for input data
    while ii < Qdta.shape[1]:
        a = np.array(dd.iloc[rctr:(rctr + 100), ii + 2:ii + 2 + ss])
        bb = np.array(np.zeros(a.shape)).reshape([1, -1])
        bb.shape

        for kk in np.arange(0, a.shape[0], step = 1):
            bb[0, kk*ss:(kk+1)*ss] = a[kk, 0:ss]  

        ii += ss
    
        cc[cctr, :] = bb
        cctr += 1
    # update row counter
    print(str(cctr) + " of " + str(cc.shape[0]))
    rctr += 100

    
print(str(np.round(time.time() - stt)) + " Seconds")

In [ ]:
plt.figure(figsize=(20,100))
plt.imshow(cc, cmap='hot', interpolation='nearest')

plt.show()

In [ ]:
# double check
for ii in np.arange(5000, 10000):
    plt.plot(cc[ii, np.arange(0, 800, step = 8)], cc[ii, np.arange(1, 800, step = 8)])

In [ ]:
# make data frame and name columns
tsDf = pd.DataFrame(cc)

In [ ]:
tsDf["timestep"] = np.floor(np.arange(0,tsDf.shape[0]) / 2500).astype(int)
tsDf.head()

In [ ]:
new_cols = ["x", "y", "theta", "phi", "x_dot", "y_dot", "theta_dot", "phi_dot"]

In [ ]:
jj = 0
hh = []
for jj in range(100):
    hh.append([new_cols[ii] +"_" +str(jj) for ii in range(8)])

allCols = np.concatenate(hh)

In [ ]:
tsDf.columns = np.append(allCols, "timestep")

In [ ]:
tsDf.head()

In [ ]:
forceAngle = np.transpose(Vmat["ValSp3"])
fa = forceAngle.reshape((250000, 12), order = "C" )
fa2=  pd.DataFrame(fa)
fa2.columns = np.append([new_cols[ii] +"_" +str(99) for ii in range(8)], ["F", "alpha", "tao", "cost"])
fa2["Fx"] = fa2.F * np.cos(fa2.alpha)
fa2["Fy"] = fa2.F * np.sin(fa2.alpha)
fa2.shape

In [ ]:
combDF = pd.merge(tsDf.dropna(), fa2.dropna(), 
                  left_on = [new_cols[ii] +"_" +str(99) for ii in range(8)], 
                  right_on = [new_cols[ii] +"_" +str(99) for ii in range(8)], 
                  how = "inner")


In [ ]:
combDF.head()

In [ ]:
combDF.shape

In [ ]:
# show how many NA's were dropped
fa2.shape[0] - fa2.dropna().shape[0]

### Finished data munging

Now fit models

### Keras model

In [ ]:
X = combDF.iloc[:, np.concatenate((np.arange(0,8), np.arange(792,800)))].copy()
Y = combDF.iloc[:, [803, 805, 806]]

# # subtract x_0 and y_0 from everything
X.x_99 = X.x_99 - X.x_0
X.y_99 = X.y_99 - X.y_0

X.x_0 = X.y_0 = 0


In [ ]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.3, random_state = 123)

In [ ]:
from sklearn.preprocessing import StandardScaler  
scalerX = StandardScaler()  
scalerY = StandardScaler()  

# Don't cheat - fit only on training data
scalerX.fit(Xtrain)  
scalerY.fit(Ytrain) 

Xtrain_scaled = scalerX.transform(Xtrain)  
Ytrain_scaled = scalerY.transform(Ytrain)  

# apply same transformation to test data
Xtest_scaled = scalerX.transform(Xtest)
Ytest_scaled = scalerY.transform(Ytest)

In [ ]:
Ytrain_scaled.shape[1]

In [ ]:
model = Sequential()
model.add(Dense(400, activation = "tanh", input_dim=Xtrain_scaled.shape[1]))
model.add(Dense(400, activation = "tanh"))
model.add(Dense(16, activation = "tanh"))
model.add(Dense(16, activation = "tanh"))
model.add(Dense(Ytrain_scaled.shape[1], activation = "linear")) # final layer has 3 outputs
model.compile(optimizer='rmsprop',loss='mse')

In [ ]:
stt = time.time()
model.fit(Xtrain_scaled, Ytrain_scaled, epochs=10000, initial_epoch=0, 
          batch_size = 100000)
print(time.time() - stt) # <10 seconds for 100 epochs

In [ ]:
nnpreds = model.predict(Xtest_scaled)

In [ ]:
plt.close("all")
fig, axs = plt.subplots(1,3, figsize=(15, 5), facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = 0.1, wspace=0.3)
fig.suptitle('Predicted vs. acutal for 4-hidden-layer Neural Network - With Keras', fontsize=14, fontweight='bold')

axs = axs.ravel()

nms = ["tau", "Fx", 'Fy']
for ii in range(3):
    # read in image
    
    try:
        axs[ii].scatter(y = Ytest_scaled[:,ii],x = nnpreds[:,ii], s = 1 )
        axs[ii].set_xlabel("Predicted Value (scaled)")
        if(ii == 0):
            axs[ii].set_ylabel("Actual Value\n(scaled)")
        axs[ii].set_title(nms[ii])
        axs[ii].plot(Ytest_scaled[:,ii], Ytest_scaled[:,ii], 'b')
    except:
        pass

    
fig.savefig(os.path.join(figDir, "Keras_zerod.png"), dpi = 120, bbox_inches='tight')

In [ ]:
os.path.join(figDir, "Keras_zerod.png")

In [ ]:
Xtrain_scaled = np.transpose(np.random.randn(1000))
Ytrain_scaled = np.transpose(np.sin(Xtrain_scaled)+ np.random.randn(1000)*0.1)
plt.scatter(Xtrain_scaled, Ytrain_scaled) 

In [ ]:
model = Sequential()

In [ ]:
model.add(Dense(units=4, activation='relu', input_dim = 1000))
model.add(Dense(1, activation='tanh'))


In [ ]:
model.layers

In [ ]:
Xtrain_scaled.shape

In [ ]:
model.compile(optimizer='rmsprop',
              loss='mse')

In [ ]:
model.fit(Xtrain_scaled, Ytrain_scaled, epochs=5)

In [ ]:
from sklearn.preprocessing import scale

# Generate dummy data
Xtrain_scaled = np.random.randn(100000, 100).reshape((-1,100))*100
Ytrain_scaled = np.sin(Xtrain_scaled[:,0]/50)*40+ np.random.randn(Xtrain_scaled.shape[0])*5
Ytrain_scaled2 = np.cos(Xtrain_scaled[:,1]/100)+ np.random.randn(Xtrain_scaled.shape[0])*0.1

Ytrain_scaled3 = np.transpose(np.vstack([Ytrain_scaled, Ytrain_scaled2]))
print(Xtrain_scaled.shape)
print(Ytrain_scaled3.shape)



scalerX = StandardScaler()  
scalerY = StandardScaler()  

scalerX.fit(Xtrain_scaled)  
scalerY.fit(Ytrain_scaled3) 

Xtrain_scaled = scalerX.transform(Xtrain_scaled)  
Ytrain_scaled4 = scalerY.transform(Ytrain_scaled3)  


model = Sequential()
model.add(Dense(100, activation = "tanh",  input_dim=Xtrain_scaled[:,0:2].shape[1]))
model.add(Dense(100, activation = "tanh"))
model.add(Dense(50, activation = "tanh"))
model.add(Dense(500, activation = "tanh"))
model.add(Dense(500, activation = "tanh"))
model.add(Dense(2))
model.compile(optimizer='rmsprop',loss='mse')

In [ ]:
plt.scatter(Xtrain_scaled[:,0], Ytrain_scaled4[:,0])
plt.scatter(Xtrain_scaled[:,1], Ytrain_scaled4[:,1])

In [ ]:
# Train the model
model.fit(Xtrain_scaled[:,0:2], Ytrain_scaled4, epochs=1000, batch_size = 10000)

In [ ]:
preds = model.predict(Xtrain_scaled[:,0:2])
plt.scatter(Xtrain_scaled[:,0], Ytrain_scaled4[:,0])
plt.scatter(Xtrain_scaled[:,0], preds[:,0])
plt.show()

plt.scatter(Xtrain_scaled[:,1], Ytrain_scaled4[:,1])
plt.scatter(Xtrain_scaled[:,1], preds[:,1])


In [ ]:
plt.scatter(Ytrain_scaled4[:,1], preds[:,1])

In [ ]:
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [ ]:
# define the model
def larger_model():
    # create model
    model = Sequential()
    model.add(Dense(100, activation = "tanh",  input_dim=Xtrain_scaled.shape[1]))
    model.add(Dense(100))
    model.add(Dense(50))
    model.add(Dense(500))
    model.add(Dense(500))
    model.add(Dense(2))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [ ]:
estimator = KerasRegressor(build_fn=larger_model, epochs=100, batch_size=10000, verbose=1)

In [ ]:
kfold = KFold(n_splits=2, random_state=123)
results = cross_val_score(estimator, Xtrain_scaled, Ytrain_scaled4, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))


In [ ]:
estimator.fit(Xtrain_scaled, Ytrain_scaled4)

In [ ]:
preds = estimator.predict(Xtrain_scaled)
plt.scatter(Xtrain_scaled[:,0], Ytrain_scaled4[:,0])
plt.scatter(Xtrain_scaled[:,0], preds[:,0])
plt.show()

### Sklearn model

In [ ]:
X = combDF.iloc[:, np.concatenate((np.arange(0,8), np.arange(792,800)))].copy()
Y = combDF.iloc[:, [803, 805, 806]]

# # subtract x_0 and y_0 from everything
X.x_99 = X.x_99 - X.x_0
X.y_99 = X.y_99 - X.y_0

X.x_0 = X.y_0 = 0


In [ ]:
Y.head()

In [ ]:
X.tail()

In [ ]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.3, random_state = 123)

In [ ]:
# refref: scale test and training separately:
from sklearn.preprocessing import StandardScaler  
scalerX = StandardScaler()  
scalerY = StandardScaler()  

# Don't cheat - fit only on training data
scalerX.fit(Xtrain)  
scalerY.fit(Ytrain) 

Xtrain_scaled = scalerX.transform(Xtrain)  
Ytrain_scaled = scalerY.transform(Ytrain)  

# apply same transformation to test data
Xtest_scaled = scalerX.transform(Xtest)
Ytest_scaled = scalerY.transform(Ytest)

In [ ]:
xcheck = scalerX.inverse_transform(Xtrain_scaled)
np.allclose(xcheck, np.array(Xtrain))

In [ ]:
# save scalers
import pickle

with open(os.path.join(dataDir, "StandardScaler_zerod.pkl"), 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump([scalerX, scalerY], f)

In [ ]:
del scalerX
del scalerY

In [ ]:
# Getting back the objects:
with open(os.path.join(dataDir, "StandardScaler_zerod.pkl"), 'rb') as f:  # Python 3: open(..., 'rb')
    scalerX, scalerY = pickle.load(f)

In [ ]:
# check scaler
plt.hist(Xtrain.iloc[:, 3])
plt.show()
plt.hist(Xtrain_scaled[:,3])
plt.show()

xcheck = scalerX.inverse_transform(Xtrain_scaled)
plt.hist(xcheck[:,3])

In [ ]:
print(Xtrain.shape,  Ytrain.shape, Xtest.shape, Ytest.shape)

In [ ]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# stt = time.time()
# mor = MultiOutputRegressor(RandomForestRegressor(500, n_jobs = 4)).fit(Xtrain_scaled, Ytrain_scaled)
# print(time.time() - stt) # ~ 13 minutes

In [ ]:
# # save RF model
# from sklearn.externals import joblib
# joblib.dump(mor, os.path.join(dataDir, "FitRF_Scaled2.pkl")) 

In [ ]:
# mpreds = mor.predict(Xtest_scaled)

In [ ]:
# plt.close("all")
# fig, axs = plt.subplots(1,3, figsize=(15, 5), facecolor='w', edgecolor='k')
# fig.subplots_adjust(hspace = 0.1, wspace=0.3)
# fig.suptitle('Predicted vs. acutal for Mulit-output Random Forest Regressor (500 trees)', fontsize=14, fontweight='bold')

# axs = axs.ravel()
# nms = ["tao", "Fx", "Fy"]


# for ii in range(3):
#     # read in image
    
#     try:
#         axs[ii].scatter(y = Ytest_scaled[:,ii],x = mpreds[:,ii], s = 1, c= 'indianred' )
#         axs[ii].set_xlabel("Predicted Value (Scaled)")
#         if(ii == 0):
#             axs[ii].set_ylabel("Actual Value\n(Scaled)")
#             axs[ii].set_xlim([-10, 5])
#         axs[ii].set_title(nms[ii])
#         axs[ii].plot(Ytest_scaled[:,ii], Ytest_scaled[:,ii], 'maroon')
#     except:
#         pass

    
# fig.savefig(os.path.join(figDir, "RFpreds_Scaled2.png"), dpi = 120, bbox_inches='tight')

## Multi-output neural network

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.multioutput import MultiOutputRegressor


In [ ]:
stt = time.time()
nn1 = MLPRegressor(solver='lbfgs', alpha=0,
                  hidden_layer_sizes=(100, 6), max_iter=10000, 
                  activation='tanh', verbose = True)
nnMod = MultiOutputRegressor(nn1).fit(Xtrain_scaled, Ytrain_scaled)
print(time.time() - stt) # takes 128 minutes on MBPro

In [ ]:
# save nn, so I can reload without retraining
from sklearn.externals import joblib
joblib.dump(nnMod, os.path.join(dataDir, "TrainedNN_scaled_zerod.pkl")) 

In [ ]:
# load trained nn

In [ ]:

# nnMod = joblib.load(os.path.join(dataDir, "TrainedNN_scaled_zerod.pkl")) 

In [ ]:
nnpreds = nnMod.predict(Xtest_scaled)

In [ ]:
plt.close("all")
fig, axs = plt.subplots(1,3, figsize=(15, 5), facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = 0.1, wspace=0.3)
fig.suptitle('Predicted vs. acutal for 2-hidden-layer Neural Network', fontsize=14, fontweight='bold')

axs = axs.ravel()


for ii in range(3):
    # read in image
    
    try:
        axs[ii].scatter(y = Ytest_scaled[:,ii],x = nnpreds[:,ii], s = 1 )
        axs[ii].set_xlabel("Predicted Value (scaled)")
        if(ii == 0):
            axs[ii].set_ylabel("Actual Value\n(scaled)")
        axs[ii].set_title(nms[ii])
        axs[ii].plot(Ytest_scaled[:,ii], Ytest_scaled[:,ii], 'b')
    except:
        pass

    
fig.savefig(os.path.join(figDir, "NNPreds2_zerod.png"), dpi = 120, bbox_inches='tight')

# Compare simulation vs. nn preds for test set!

In [ ]:
s1 = pd.concat([Xtest, Ytest], axis = 1).reset_index(drop = True)

shuff2 = s1

# predictions
XX_scaled = scalerX.transform(shuff2.iloc[:, 0:16])
nnpreds3 = nnMod.predict(XX_scaled)

# add preds back to data frame
nn_back = scalerY.inverse_transform(nnpreds3)
preds1 = pd.DataFrame(nn_back)
preds1.columns = ["tao_pred", "Fx_pred", "Fy_pred"]
preds1["F_pred"] = np.sqrt(preds1.Fx_pred **2 + preds1.Fy_pred **2)


pre_alpha = np.arctan2(preds1.Fy_pred, preds1.Fx_pred )
alpha = pre_alpha.copy()

for ii in pre_alpha.index:
    if (preds1.Fy_pred[ii] > 0) :
        alpha[ii] = pre_alpha[ii]
    else:
        alpha[ii] = (2*np.pi) + pre_alpha[ii]
        
preds1["alpha_pred"] = alpha


preds1.head()

In [ ]:
s2_1 = pd.concat([shuff2, preds1], axis = 1)
s2_1.head()

In [ ]:
s2_1.to_csv(os.path.join(dataDir, "simulationData5_withFinal_testset.csv"), index=False)
print(dataDir)

simSet1 = s2_1.loc[:, ['x_0', 'y_0', 'theta_0', 'phi_0', 'x_dot_0', 'y_dot_0', 'theta_dot_0',
       'phi_dot_0', 'tao_pred','F_pred','alpha_pred']]


simSet1.to_csv(os.path.join(dataDir, "simulationDataset5_blinded_testset.csv"), index=False)
simSet1.head()

In [ ]:
simSet1.shape

___

In [ ]:
baseDir

In [ ]:
# refref: run simulation vs. nn preds for test set!


___
### Create dataset with same initial conditions and shuffle output conditions

In [ ]:
s1 = combDF.iloc[:, np.hstack([np.arange(0,8),np.arange(792,800), [801, 802, 803]])].sample(1000, random_state = 1234)
s1.reset_index(inplace = True)

# set x_0 and y_0 to (0,0)  -- refref: may want to do this for training, actually
s1.x_99 = s1.x_99 - s1.x_0
s1.y_99 = s1.y_99 - s1.y_0

s1.x_0 = 0
s1.y_0 = 0

s1.head()

In [ ]:
s1.columns

In [ ]:
sTemp = s1.loc[:, ['x_99', 'y_99', 'theta_99', 'phi_99',
       'x_dot_99', 'y_dot_99', 'theta_dot_99', 'phi_dot_99']]
sInit = s1.loc[:, ['index', 'x_0', 'y_0', 'theta_0', 'phi_0', 'x_dot_0', 'y_dot_0',
       'theta_dot_0', 'phi_dot_0']]

shuff = sTemp.sample(sTemp.shape[0], random_state = 12346)
shuff2 = pd.concat([sInit, shuff.reset_index(drop = True)], axis = 1)



shuff2.tail()

In [ ]:
# predictions
XX_scaled = scalerX.transform(shuff2.iloc[:, 1:17])
nnpreds3 = nnMod.predict(XX_scaled)

# add preds back to data frame
nn_back = scalerY.inverse_transform(nnpreds3)
preds1 = pd.DataFrame(nn_back)
preds1.columns = ["tao_pred", "Fx_pred", "Fy_pred"]
preds1["F_pred"] = np.sqrt(preds1.Fx_pred **2 + preds1.Fy_pred **2)


pre_alpha = np.arctan2(preds1.Fy_pred, preds1.Fx_pred )
alpha = pre_alpha.copy()

for ii in pre_alpha.index:
    if (preds1.Fy_pred[ii] > 0) :
        alpha[ii] = pre_alpha[ii]
    else:
        alpha[ii] = (2*np.pi) + pre_alpha[ii]
        
preds1["alpha_pred"] = alpha


preds1.head()

In [ ]:
s2_1 = pd.concat([shuff2, preds1], axis = 1)
s2_1.head()

In [ ]:
s2_1.to_csv(os.path.join(dataDir, "simulationData4_withFinal.csv"), index=False)
print(dataDir)

simSet1 = s2_1.loc[:, ['x_0', 'y_0', 'theta_0', 'phi_0', 'x_dot_0', 'y_dot_0', 'theta_dot_0',
       'phi_dot_0', 'tao_pred','F_pred','alpha_pred']]


simSet1.to_csv(os.path.join(dataDir, "simulationDataset4_blinded.csv"), index=False)
simSet1.head()

In [ ]:
# refref: might get better predictions if we set initial X and Y to (0,0) and
# and transform output X and Y relative to the start

--- Dataset to check in Jorge's simulation ---

In [ ]:
## generate dataset for Jorge to send to model
s1 = combDF.iloc[:, np.hstack([np.arange(0,8),np.arange(792,800), [801, 802, 803]])].sample(1000)
s1.head()

In [ ]:
# predictions
XX_scaled = scalerX.transform(s1.iloc[:, 0:16])
nnpreds2 = nnMod.predict(XX_scaled)

In [ ]:
s1.iloc[:, 0:16].shape

In [ ]:
s1["Fx"] = s1.F * np.cos(s1.alpha)
s1["Fy"] = s1.F * np.sin(s1.alpha)
s1.head()

In [ ]:
# compute F, alpha, to double check
s1["F_"] = np.sqrt(s1.Fx **2 + s1.Fy **2)


pre_alpha = np.arctan2(s1.Fy, s1.Fx )
alpha = pre_alpha.copy()

for ii in pre_alpha.index:
    if (s1.Fy[ii] > 0) :
        alpha[ii] = pre_alpha[ii]
    else:
        alpha[ii] = (2*np.pi) + pre_alpha[ii]
        
s1["alpha_"] = alpha
s1.head()

In [ ]:
YY_scaled = scalerY.transform(s1.iloc[:, [18, 19, 20]])
pd.DataFrame(YY_scaled).head()

In [ ]:
# plot to double-check
plt.close("all")
fig, axs = plt.subplots(1,3, figsize=(15, 5), facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = 0.1, wspace=0.3)
fig.suptitle('Predicted vs. acutal for subset', fontsize=14, fontweight='bold')

axs = axs.ravel()

# convert back to original values
nn_back = scalerY.inverse_transform(nnpreds2)
YY_orig = s1.iloc[:, [18, 19, 20]]


for ii in range(3):
    # read in image
    
    try:
        axs[ii].scatter(y = YY_orig.iloc[:,ii],x = nn_back[:,ii], s = 1 )
        axs[ii].set_xlabel("Predicted Value (scaled)")
        if(ii == 0):
            axs[ii].set_ylabel("Actual Value\n(scaled)")
        axs[ii].set_title(nms[ii])
        axs[ii].plot(nn_back[:,ii], nn_back[:,ii], 'b')
    except:
        pass

In [ ]:
# add predictions to data frame
preds1 = pd.DataFrame(nn_back)
preds1.columns = ["tao_pred", "Fx_pred", "Fy_pred"]
preds1.head()

In [ ]:
cc2 = pd.concat([s1.reset_index(drop=True), preds1.reset_index(drop=True)], axis = 1)
cc2.head()

In [ ]:
cc2["F_pred"] = np.sqrt(cc2.Fx_pred **2 + cc2.Fy_pred **2)


pre_alpha = np.arctan2(cc2.Fy_pred, cc2.Fx_pred )
alpha = pre_alpha.copy()

for ii in pre_alpha.index:
    if (cc2.Fy_pred[ii] > 0) :
        alpha[ii] = pre_alpha[ii]
    else:
        alpha[ii] = (2*np.pi) + pre_alpha[ii]
        
cc2["alpha_pred"] = alpha

In [ ]:
cc2.head()

In [ ]:
cc2.columns

In [ ]:
cc2.to_csv(os.path.join(dataDir, "simulationData1_withFinal.csv"), index=False)
print(dataDir)

In [ ]:
simSet1 = cc2.loc[:, ['x_0', 'y_0', 'theta_0', 'phi_0', 'x_dot_0', 'y_dot_0', 'theta_dot_0',
       'phi_dot_0', 'tao_pred','F_pred','alpha_pred']]
simSet1.head()

In [ ]:
simSet1.to_csv(os.path.join(dataDir, "simulationDataset1_blinded.csv"), index=False)
print(dataDir)

--- Dataset #2 ---

In [ ]:
## now build another training dataset, that is random combinations of initial values

def shuffle(df, n=1, axis=0):     
    df = df.copy()
    for _ in range(n):
        df.apply(np.random.shuffle, axis=axis)
    return df

In [ ]:
ss = combDF.iloc[:, np.hstack([np.arange(0,8),np.arange(792,800)])].sample(1500)
ss.reset_index(drop = True, inplace = True)
ss.head()

In [ ]:
s2 = shuffle(ss)
s2.head()

In [ ]:
# predict on shuffled data
s2_scaled = scalerX.transform(s2)
nnpreds3 = nnMod.predict(s2_scaled)

In [ ]:
# add preds back to data frame
nn_back = scalerY.inverse_transform(nnpreds3)
preds1 = pd.DataFrame(nn_back)
preds1.columns = ["tao_pred", "Fx_pred", "Fy_pred"]
preds1["F_pred"] = np.sqrt(preds1.Fx_pred **2 + preds1.Fy_pred **2)


pre_alpha = np.arctan2(preds1.Fy_pred, preds1.Fx_pred )
alpha = pre_alpha.copy()

for ii in pre_alpha.index:
    if (preds1.Fy_pred[ii] > 0) :
        alpha[ii] = pre_alpha[ii]
    else:
        alpha[ii] = (2*np.pi) + pre_alpha[ii]
        
preds1["alpha_pred"] = alpha


preds1.head()

In [ ]:
s2_1 = pd.concat([s2, preds1], axis = 1)
s2_1.head()

In [ ]:
s2_1.to_csv(os.path.join(dataDir, "simulationData2_withFinal.csv"), index=False)

simSet2 = s2_1.loc[:, ['x_0', 'y_0', 'theta_0', 'phi_0', 'x_dot_0', 'y_dot_0', 'theta_dot_0',
       'phi_dot_0', 'tao_pred','F_pred','alpha_pred']]

simSet2.to_csv(os.path.join(dataDir, "simulationDataset2_blinded.csv"), index=False)
simSet2.head()

___

___ Dataset 3 -- Sample initial and get randomized final states 

#
#
#


___

In [ ]:
ss = combDF.iloc[:, np.hstack([np.arange(0,8),np.arange(792,800)])].sample(1200)
ss.reset_index(drop = True, inplace = True)
ss.head()

In [ ]:
# only mix up possible output states
s3_1 = shuffle(ss.iloc[:, 8:])
s3_2 = pd.concat([ss.iloc[:,0:8], s3_1], axis = 1)
s3_2.head()

In [ ]:
# predict on shuffled data
s3_scaled = scalerX.transform(s3_2)
nnpreds3_1 = nnMod.predict(s3_scaled)

In [ ]:
# add preds back to data frame
nn_back = scalerY.inverse_transform(nnpreds3_1)
preds1 = pd.DataFrame(nn_back)
preds1.columns = ["tao_pred", "Fx_pred", "Fy_pred"]
preds1["F_pred"] = np.sqrt(preds1.Fx_pred **2 + preds1.Fy_pred **2)


pre_alpha = np.arctan2(preds1.Fy_pred, preds1.Fx_pred )
alpha = pre_alpha.copy()

for ii in pre_alpha.index:
    if (preds1.Fy_pred[ii] > 0) :
        alpha[ii] = pre_alpha[ii]
    else:
        alpha[ii] = (2*np.pi) + pre_alpha[ii]
        
preds1["alpha_pred"] = alpha


preds1.head()

In [ ]:
s3_4 = pd.concat([s3_2, preds1], axis = 1)
s3_4.head()

In [ ]:
s3_4.to_csv(os.path.join(dataDir, "simulationData3_withFinal.csv"), index=False)

simSet3 = s3_4.loc[:, ['x_0', 'y_0', 'theta_0', 'phi_0', 'x_dot_0', 'y_dot_0', 'theta_dot_0',
       'phi_dot_0', 'tao_pred','F_pred','alpha_pred']]

simSet3.to_csv(os.path.join(dataDir, "simulationDataset3_blinded.csv"), index=False)
simSet3.head()

In [ ]:
## nn practice

Xx = np.transpose(np.array([np.arange(-10, 10, step = 0.001), np.random.uniform(size = 20000)]))
Xx_scaled = sklearn.preprocessing.scale(Xx, axis = 0)
Yy = np.sin(Xx[:,0]) *4 **2 + 50
Yy_scaled = sklearn.preprocessing.scale(Yy, axis = 0)
plt.plot(Xx_scaled[:,0], Yy)
plt.show()
plt.scatter(Xx_scaled[:,1], Yy, s = 1)

In [ ]:
plt.hist(Xx_scaled[:,1])

In [ ]:
stt = time.time()
nn1 = MLPRegressor(solver='lbfgs', alpha=0,
                  hidden_layer_sizes=(10,10), max_iter=1000, 
                  verbose = True, tol=0.0000100,
                  activation='tanh')
nnMod = nn1.fit(Xx_scaled, Yy)
print(time.time() - stt)

In [ ]:
nnpreds = nnMod.predict(Xx_scaled)

In [ ]:
plt.scatter(x = Xx_scaled[:,0], y = Yy)
plt.scatter(x = Xx_scaled[:,0], y = nnpreds)

In [ ]:
plt.scatter(x = Xx_scaled[:,0], y = Yy_scaled)
plt.scatter(x = Xx_scaled[:,0], y = nnpreds)

In [ ]:
plt.scatter(x = nnpreds, y = Yy)

In [ ]:
# merge with F, alpha, tao
Vdta = Vmat["ValSp3"]
dd2 = pd.DataFrame(Vdta)
dd2.columns = np.append([new_cols[ii] +"_" +str(99) for ii in range(8)], ["F", "alpha", "tao"])
print(dd2.shape)
dd2.head()

In [ ]:
ff = dd


stp = 8
for stp in np.arange(10):
    xx = 1
    yy = 2
    for ii in np.arange(1000):
        plt.scatter(ff.iloc[stp*100:(stp+1)*100,xx], ff.iloc[stp*100:(stp+1)*100, yy], s = 1)
        xx += 8
        yy +=8
    plt.show()


In [ ]:
ii = 200

for jj in np.arange(0, dta.shape[0], step = 8):
    plt.scatter(dta[ii:ii+100, jj], dta[ii:ii+100, jj + 1], s = 1)

In [ ]:
new_cols = ["x", "y", "theta", "phi", "x_dot", "y_dot", "theta_dot", "phi_dot"]

In [ ]:
# another possible way to rearrange the data
ss = 8
a = dta
bb = np.array(np.zeros(a.shape)).reshape([-1, ss])
bb.shape

In [ ]:
a.shape[1]
a.shape[0]
pd.DataFrame(a[0:10, :])

In [ ]:
for kk in np.arange(0, a.shape[1], step = a.shape[0]):
    bb[kk:(kk+a.shape[0]),0:ss] = a[0:, kk:(ss+kk)]  

    
cc = pd.DataFrame(bb)
cc.columns = new_cols
cc

In [ ]:
for ii in np.arange(0, 10000, step = 100):
    plt.scatter(cc.loc[ii:ii+100, "x"],cc.loc[ii:ii+100, "y"], s = 1)

In [ ]:
dd2 = dta.reshape((-1, 8))
dd3 = pd.DataFrame(dd2)

In [ ]:
dd3.columns =  ["x", "y", "theta", "phi", "x_dot", "y_dot", "theta_dot", "phi_dot"]
dd3.head()

plt.scatter(dd3.loc[1:10000, "x"],dd3.loc[1:10000, "y"], s = 1 )

In [ ]:
dta.shape[1]

In [ ]:
new_cols = ["x", "y", "theta", "phi", "x_dot", "y_dot", "theta_dot", "phi_dot"]

In [ ]:
initialCond = dta[np.arange(0, dta.shape[0], step = 100),: ]
finalCond = dta[np.arange(99, dta.shape[0], step = 100), :]

In [ ]:
# reshape data
i2 = initialCond.reshape((250000, 8))
f2 = finalCond.reshape((250000, 8))

In [ ]:
#pd.DataFrame(i2[2495:2555, ])

In [ ]:
conddta = np.hstack((i2, f2))
condDta2 = pd.DataFrame(conddta)
condDta2.columns = np.concatenate(([ii + "_i" for ii in new_cols], [ii + "" for ii in new_cols]))
print(condDta2.shape)
condDta2.head()

In [ ]:
Vmat = scipy.io.loadmat(os.path.join(dataDir, "ValSp3_1_am_con.mat"))
Vmat

In [ ]:
# this might work
forceAngle = np.transpose(Vmat["ValSp3"])
fa = forceAngle.reshape((250000, 12), order = "C" )
fa2=  pd.DataFrame(fa)
fa2.columns = np.concatenate((new_cols, ["F", "alpha", "tao", "cost"]))
fa2["Fx"] = fa2.F * np.cos(fa2.alpha)
fa2["Fy"] = fa2.F * np.sin(fa2.alpha)
fa2
combDF = pd.merge(condDta2.dropna(), fa2.dropna(), left_on = new_cols, right_on = new_cols, how = "inner")


In [ ]:
combDF

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
combDF.dropna().iloc[:, 0:16]

In [ ]:
# drop NA's
X = np.array(combDF.dropna().iloc[:, 0:16])
Y = np.array(combDF.dropna().iloc[:, [18, 20, 21]])

# check: refref: not checked
print(all(X[:,9] == Y[:,1]))
print(len(X))

#Y = Y[:, [10, 12, 13]]
Y.shape

In [ ]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.3)

In [ ]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor

mor = MultiOutputRegressor(RandomForestRegressor(1000, n_jobs = 4)).fit(X, Y)

In [ ]:
mpreds = mor.predict(Xtest)

In [ ]:

nms = ["tao", "Fx", "Fy"]

for ii in range(3):
    plt.scatter(y = Ytest[:,ii],x = mpreds[:,ii], s = 2 )
    plt.xlabel("Predicted Value")
    plt.ylabel("Actual Value")
    plt.title(nms[ii])
    plt.show()


In [ ]:
combDF.head()

In [ ]:
a = np.vstack((np.arange(8), np.arange(7, 15), np.arange(7, 15)))
a

In [ ]:
ss = 2
bb = np.array(np.zeros(a.shape)).reshape([-1, ss])
bb

In [ ]:
np.arange(0, a.shape[1], step = 2)

In [ ]:
a.shape[0]

In [ ]:
ss = 2

for kk in np.arange(0, a.shape[1], step = a.shape[0]):
    bb[kk:(kk+a.shape[0]),0:ss] = a[0:, kk:(ss+kk)]  
bb

In [ ]:
bb[kk:(kk+ss),0:ss]

In [ ]:
ss =2
kk = 2

a[0:, kk:(ss+kk)]
bb[kk:(kk+a.shape[0]),0:ss]



In [ ]:
pd.DataFrame(a)

In [ ]:
# refref: figure out how to reshape this

a.reshape((-1, 2), order = "F")

In [ ]:
np.reshape?

In [ ]:
dd = pd.DataFrame(fa)
print(dd.shape)
dd.head()

In [ ]:
fa = dd.transpose()
fa.columns = np.concatenate((new_cols, ["F", "alpha", "tao", "cost"]))
print(fa.shape)
fa.head()

In [ ]:
# make Fx and Fy
fa["Fx"] = fa.F * np.cos(fa.alpha)
fa["Fy"] = fa.F * np.sin(fa.alpha)

In [ ]:
for ii in np.arange(1, spray.shape[1], step = 8):

    tmparr = np.hstack((sparray[0, ii:ii+8], sparray[99, ii:ii+8]))

    if ii == 1:
        newArr = tmparr
    else: 
        newArr = np.vstack((newArr, tmparr))

In [ ]:
newDF = pd.DataFrame(newArr)
newDF.head()

In [ ]:
newDF.columns = np.concatenate(([ii + "_i" for ii in new_cols], [ii + "" for ii in new_cols]))

In [ ]:
print(newDF.shape)
newDF.head()

In [ ]:
# drop NA's
X = np.array(newDF.dropna())
Y = np.array(fa.dropna())

# check
print(all(X[:,9] == Y[:,1]))
print(len(X))

Y = Y[:, [10, 12, 13]]
Y.shape

In [ ]:
Y[0:10, ]

In [ ]:
combDF = pd.merge(newDF.dropna(), fa.dropna(), left_on = new_cols, right_on = new_cols, how = "inner")
print(combDF.shape)
combDF.head()

In [ ]:
# split into test and training set
from sklearn.model_selection import train_test_split

In [ ]:
import tensorflow as tf

In [ ]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.3)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

clf1 = RandomForestRegressor(1000, n_jobs = 4).fit(Xtrain, Ytrain)
Ypred1 = clf1.predict(Xtest)

fig, ax = plt.subplots()
ax.plot(Ypred1[:, 0], Ypred1[:, 1], 'o', alpha=0.5)
ax.set_xlabel('$y_1$'); ax.set_ylabel('$y_2$');

In [ ]:
for ii in range(3):
    plt.scatter(Ytest[:,ii],Ypred1[:,ii] )
    plt.show()

In [ ]:
class MultiOutputRF(object):
    
    '''From here: http://astrohackweek.org/blog/multi-output-random-forests.html'''
    def __init__(self, *args, **kwargs):
        self.args = args
        self.kwargs = kwargs
        
    def fit(self, X, Y):
        X, Y = map(np.atleast_2d, (X, Y))
        assert X.shape[0] == Y.shape[0]
        Ny = Y.shape[1]
        
        self.clfs = []
        for i in range(Ny):
            clf = RandomForestRegressor(*self.args, **self.kwargs)
            Xi = np.hstack([X, Y[:, :i]])
            yi = Y[:, i]
            self.clfs.append(clf.fit(Xi, yi))
            
        return self
        
    def predict(self, X):
        Y = np.empty([X.shape[0], len(self.clfs)])
        for i, clf in enumerate(self.clfs):
            Y[:, i] = clf.predict(np.hstack([X, Y[:, :i]]))
        return Y

In [ ]:
clf2 = MultiOutputRF(1000, n_jobs = 4).fit(Xtrain, Ytrain)
Ypred2 = clf2.predict(Xtest)

In [ ]:
# note: might want to break down into Fx and Fy, b/c radians don't work well with MSE

nms = ["tao", "Fx", "Fy"]

for ii in range(3):
    plt.scatter(y = Ytest[:,ii],x = Ypred2[:,ii], s = 2 )
    plt.xlabel("Predicted Value")
    plt.ylabel("Actual Value")
    plt.title(nms[ii])
    plt.show()

In [ ]:
# this works surprisingly well

from sklearn.multioutput import MultiOutputRegressor

mor = MultiOutputRegressor(RandomForestRegressor(1000, n_jobs = 4)).fit(X, Y)

In [ ]:
mpreds = mor.predict(Xtest)

In [ ]:
nms = ["tao", "Fx", "Fy"]

for ii in range(3):
    plt.scatter(y = Ytest[:,ii],x = mpreds[:,ii], s = 2 )
    plt.xlabel("Predicted Value")
    plt.ylabel("Actual Value")
    plt.title(nms[ii])
    plt.show()

In [ ]:
# plot residuals
nms = ["tao", "Fx", "Fy"]

for ii in range(3):
    plt.scatter(y = Ytest[:,ii]- mpreds[:,ii], x = np.arange(len(Ytest[:,ii])), s = 2 )
    plt.xlabel("Predicted Value")
    plt.ylabel("Actual Value")
    plt.title(nms[ii])
    plt.show()

In [ ]:
# refref: next step -- neural network
#from sklearn import cross_validation
#from sklearn.neural_network import MLPRegressor

#model = MLPRegressor(solver='lbfgs',alpha=0.001,hidden_layer_sizes=(150,)) 
#cross_validation.cross_val_score(model, X, Y, scoring='mean_squared_error')

In [ ]:
# another possible way to rearrange the data
ss = 12
a = np.transpose(Vmat["ValSp3"])
bb = np.array(np.zeros(a.shape)).reshape([-1, ss])
bb

for kk in np.arange(0, a.shape[1], step = a.shape[0]):
    bb[kk:(kk+a.shape[0]),0:ss] = a[0:, kk:(ss+kk)]  

    
forceAngle = pd.DataFrame(bb)
forceAngle.columns = np.concatenate((new_cols, ["F", "alpha", "tao", "cost"]))
forceAngle